# Sur Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importation des bibliothèques essentielles

In [2]:
import numpy as np
import pandas as pd

import re
import nltk
import unicodedata

# Exploration de données

In [3]:
df = pd.read_csv('nips.csv')
df = df[df['Abstract'] != "Abstract Missing"]
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4933 entries, 0 to 4932
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Year      4933 non-null   int64 
 1   Title     4933 non-null   object
 2   Abstract  4933 non-null   object
dtypes: int64(1), object(2)
memory usage: 115.7+ KB


In [6]:
df = pd.read_csv('/content/drive/My Drive/nips.csv')
df = df[df['Abstract'] != "Abstract Missing"]
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4933 entries, 0 to 4932
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Year      4933 non-null   int64 
 1   Title     4933 non-null   object
 2   Abstract  4933 non-null   object
dtypes: int64(1), object(2)
memory usage: 115.7+ KB


In [7]:
text = df['Abstract']
text.head()

,Abstract
0,Up-propagation is an algorithm for inverting ...
1,We have constructed an inexpensive video based...
2,Non-negative matrix factorization (NMF) has pr...
3,Spike-triggered averaging techniques are effec...
4,"We consider continuous state, continuous actio..."


In [8]:
corpus_init = ' '.join(list(text))
words_init = corpus_init.split()
n_words_init = len(words_init)
unique_words_init = sorted(list(set(words_init)))
n_unique_words_init = len(unique_words_init)
print("Total number of words before text preprocessing:", n_words_init)
print("Total number of unique words before text preprocessing: ", n_unique_words_init)
print(unique_words_init[:100])
print(unique_words_init[100:200])
print(unique_words_init[200:300])

Total number of words before text preprocessing: 732614
Total number of unique words before text preprocessing:  41882
['"', '"ALBO"', '"Air', '"Air\'\'', '"Answerer', '"DIRECT"', '"DUOL"', '"Expansion-Constrained', '"Generalized', '"Ghost', '"Graph', '"GuessWhat?!".', '"Hedge"', '"Hey', '"I', '"Ising\'\'', '"MNIST', '"Object', '"PixelGAN', '"Self-Expressiveness', '"Short-Dot"', '"TILT\'\'', '"Ugliness-in-Averageness"', '"What', '"additional', '"anti-Bayesian"', '"autotags")', '"averagers,"', '"batch"', '"best', '"body\'\'', '"body\'\',', '"building', '"bus', '"calibration', '"catalyst"', '"chill"', '"comparison', '"condition', '"context",', '"convergence', '"cooperative', '"correctness"', '"date"', '"deep', '"degrees', '"deltas",', '"denoise"', '"describing"', '"determinantal', '"disagreement', '"disappearance"', '"discriminating"', '"discriminative', '"early', '"edge', '"efficient"', '"em', '"equalized', '"exemplars"', '"expected', '"extended', '"external"', '"fair."', '"few-shot\'\'

# Prétraitement de texte :
Le but de cette étape est de préparer les données textuelles en les nettoyant et en les formatant de manière à ce qu'elles puissent être efficacement utilisées par le LLM.

La fonction remove_non_ascii prend en entrée une liste de mots tokenisés et supprime les caractères non-ASCII de chaque mot. Elle utilise la normalisation Unicode NFKD (Normalization Form KC) pour décomposer les caractères accentués ou spéciaux en caractères de base, puis les encode en ASCII en ignorant les caractères non valides. Après cette étape, les mots sont décodés en UTF-8 pour obtenir une version "nettoyée" sans caractères non-ASCII

In [9]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word != ',' or word != '.':
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

La fonction clean_text est utilisée pour effectuer un prétraitement détaillé sur un texte. Voici les principales étapes :

Suppression du bruit : Les URLs (liens), les références de citations (ex: \cite{}), et les contenus entre crochets, parenthèses ou accolades sont supprimés ou remplacés par des mots-clés comme "link" ou "cite". Cela permet d'enlever les éléments qui n'ont pas d'importance pour l'analyse textuelle.

Normalisation :

Le texte est converti en minuscules pour éviter toute distinction entre les majuscules et les minuscules.
Les mots avec des tirets ou des barres obliques sont séparés pour faciliter l'analyse.
Les signes de ponctuation et autres caractères non alphanumériques sont retirés.
Les numéros sont remplacés par le mot "NUMBER".
Suppression des mots non-ASCII : Les caractères non-ASCII sont éliminés à l'aide de la fonction remove_non_ascii, qui nettoie les mots.

Séparation des ponctuations : Les signes de ponctuation comme les points (.) et les virgules (,) sont séparés des mots pour être traités comme des entités distinctes.

Conversion de l'anglais britannique en anglais américain : Certains mots d'orthographe britannique sont remplacés par leur équivalent américain, par exemple "analysed" devient "analyzed" et "favourable" devient "favorable".

Cette fonction prépare donc le texte en supprimant les éléments non pertinents et en uniformisant les termes pour une meilleure compréhension et une analyse plus cohérente par les modèles de traitement du langage naturel.

In [13]:
def clean_text(text):
  # Suppression du bruit
  text = re.sub(r'\bhttps?://\w+.+[^ ]\b', 'link', text)
  text = re.sub(r'[a-zA-Z0-9]*\.?github\.?[a-zA-Z0-9]*','github',text)
  text = re.sub(r'\~\\cite\{[^}]*\}','cite',text)
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\([^)]*\)', '', text)
  text = re.sub(r'\{[^)]*\}', '', text)

  # Normalisation
  text = text.lower()  # Convertir en minuscules

  text = re.sub(r'\-',' ', text)  # Séparer les mots comme 'video-related'
  text = re.sub(r'\/',' ', text)  # Séparer les mots comme 'descriptors/tags'
  text = re.sub(r'[^a-zA-Z0-9\s\.\,]', '', text)  # Supprimer la ponctuation
  text = re.sub(r'seq2seq', 'seqtoseq', text)

  text = re.sub(r'[-+]?\d*\.?\d+', 'NUMBER', text)  # Remplacer les nombres par "NUMBER"

  text = re.sub(r'\.{2,}', '', text)  # Supprimer '..','...'
  text = re.sub(r'\.', ' . ', text)  # Séparer '.' du texte
  text = re.sub(r'\,' , ' , ', text)  # Séparer ',' du texte

  text = ' '.join(remove_non_ascii(text.split()))  # Supprimer les mots non-ASCII
  text = re.sub(r'[\w]*NUMBER[\w]*', 'NUMBER', text)  # Remplacer tout mot contenant "NUMBER" par "NUMBER"

  # Conversion de l'anglais britannique en anglais américain
  text = re.sub(r'modelled', 'modeled', text)
  text = re.sub(r'modelling', 'modeling', text)
  text = re.sub(r'parallelisation', 'parallelization', text)
  text = re.sub(r'parallelising', 'parallelizing', text)
  text = re.sub(r'analysed', 'analyzed', text)
  text = re.sub(r'generalised', 'generalized', text)
  text = re.sub(r'maximisation', 'maximization', text)
  text = re.sub(r'recogniser', 'recognizer', text)
  text = re.sub(r'optimised', 'optimized', text)
  text = re.sub(r'analyse', 'analyze', text)
  text = re.sub(r'generalisation', 'generalization', text)
  text = re.sub(r'generalised', 'generalized', text)
  text = re.sub(r'factorisation', 'factorization', text)
  text = re.sub(r'behaviour', 'behavior', text)
  text = re.sub(r'interpretted', 'interpreted', text)
  text = re.sub(r'neighbouring', 'neighboring', text)
  text = re.sub(r'neighbour', 'neighbor', text)
  text = re.sub(r'neighbours', 'neighbors', text)
  text = re.sub(r'dependant', 'dependent', text)
  text = re.sub(r'localisation', 'localization', text)
  text = re.sub(r'amortised', 'amortized', text)
  text = re.sub(r'amortisation', 'amortization', text)
  text = re.sub(r'neutralising', 'neutralizing', text)
  text = re.sub(r'prioritised', 'prioritized', text)
  text = re.sub(r'characterised', 'characterized', text)
  text = re.sub(r'characterise', 'characterize', text)
  text = re.sub(r'centeralised', 'centeralized',text)
  text = re.sub(r'initialisation', 'initialization', text)
  text = re.sub(r'initialised', 'initialized', text)
  text = re.sub(r'regularisation', 'regularization', text)
  text = re.sub(r'regularised', 'regularized', text)
  text = re.sub(r'optimisation', 'optimization', text)
  text = re.sub(r'optimise', 'optimize', text)
  text = re.sub(r'minimisation', 'minimization', text)
  text = re.sub(r'generalises', 'generalizes', text)
  text = re.sub(r'parameterised', 'parameterized', text)
  text = re.sub(r'parameterises', 'parameterizes', text)
  text = re.sub(r'reparameterisation', 'reparameterization', text)
  text = re.sub(r'optimising', 'optimizing', text)
  text = re.sub(r'favourable', 'favorable', text)
  text = re.sub(r'hypothesised', 'hypothesized', text)
  text = re.sub(r'summarise', 'summarize', text)
  text = re.sub(r'standardised', 'standardized', text)
  text = re.sub(r'randomisation', 'randomization', text)
  text = re.sub(r'synchronisation', 'synchronization', text)
  text = re.sub(r'travelling', 'traveling', text)

  return text


In [14]:
text = text.apply(clean_text)

In [15]:
text_list = list(text)
corpus = ' '.join(text_list)
words = corpus.split()
n_words = len(words)
unique_words = sorted(list(set(words)))
n_unique_words = len(unique_words)
print("Total number of words:", n_words)
print("Total number of unique words: ", n_unique_words)
print(unique_words[:100])
print(unique_words[100:200])
print(unique_words[200:300])
print(unique_words[300:400])
print(unique_words[400:500])
print(unique_words[500:600])
print(unique_words[600:700])

Total number of words: 783672
Total number of unique words:  15274
[',', '.', 'a', 'aa', 'aaai', 'aalen', 'aaronson', 'ab', 'abandons', 'abbe', 'abc', 'abdominal', 'aberrant', 'abf', 'abilities', 'ability', 'abilityto', 'ablation', 'able', 'ables', 'abnormal', 'abnormalities', 'abnormality', 'abound', 'abounds', 'about', 'above', 'abovethreshold', 'abp', 'abrupt', 'abscissa', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'absorbed', 'absorbing', 'absorption', 'abstain', 'abstaining', 'abstention', 'abstract', 'abstracted', 'abstraction', 'abstractions', 'abstractionthe', 'abstractive', 'abstracts', 'abundance', 'abundancy', 'abundant', 'abuse', 'ac', 'academic', 'academics', 'accelerate', 'accelerated', 'accelerates', 'accelerating', 'acceleration', 'accelerators', 'accelerometers', 'accentuated', 'accept', 'acceptability', 'acceptable', 'acceptably', 'acceptance', 'accepted', 'accepts', 'access', 'accessed', 'accesses', 'accessibility', 'accessible', 'accessing', 'accident'

# Sauvegarde du texte nettoyé

In [17]:
text_str = '\n'.join(text_list)
with open("nips_clean.txt", 'w') as file:
  file.write(text_str)